In [1]:
import openai
import os
import json

from qdrant_client import QdrantClient
from qdrant_client.models import Filter, FieldCondition, MatchValue

from langsmith import Client

In [2]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [3]:
all_points = qdrant_client.scroll(
    collection_name="Amazon-items-collection-00",
    limit=100,
    offset=None,
    with_payload=True,
    with_vectors=False
)

In [4]:
all_points

([Record(id=0, payload={'description': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Na

In [5]:
all_points[0][0].payload

{'description': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Native Palm Rejection]- R

In [6]:
all_context = [{"id": data.payload["parent_asin"], "text": data.payload["description"]} for data in all_points[0]]
all_context

[{'id': 'B0BF18F6R7',
  'text': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, taking notes, and marking up PDFs, as easy and natural as a real pencil [Native Pa

#### Render a prompt to generate synthetic Eval reference dataset

In [7]:
output_schema = {
    "type": "array",
    "items": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "Suggested question.",
            },
            "chunk_ids": {
                "type": "array",
                "items": {
                    "type": "string",
                    "description": "ID of the chunk that could be used to answer the question.",
                },
            },
            "answer_example": {
                "type": "string",
                "description": "Suggested answer grounded in the context.",
            },
            "reasoning": {
                "type": "string",
                "description": "Reasoning why the question could be answered with the chunks.",
            },
        },
    },
}


SYSTEM_PROMPT = f"""
I am building a RAG application. I have a collection of 50 chunks of text.
The RAG application will act as a shopping assistant that can answer questions about the stock of the products we have available.
I will provide all of the available products to you with IDs of each chunk.
I want you to come up with 30 questions to which the answers could be grounded in the chunk context.
As an output I need you to provide me the list of questions and the IDs of the chunks that could be used to answer them.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose the chunks to answer the questions.
Construct 10 that could use multiple chunks in the answer.
Construct 15 questions that could use single chunk in the answer.
Also, include 5 questions that can't be answered with the available chunks.

<OUTPUT JSON SCHEMA>
{json.dumps(output_schema, indent=2)}
</OUTPUT JSON SCHEMA>

I need to be able to parse the json output.
"""

USER_PROMPT = f"""
Here is the list of chunks, each list element is a dictionary with id and text:
{all_context}
"""

In [8]:
print(USER_PROMPT)


Here is the list of chunks, each list element is a dictionary with id and text:
[{'id': 'B0BF18F6R7', 'text': 'KEEPRO Pencil 2nd Generation for iPad, Magnetic Wireless Charge Tilt Sensitivity Palm Rejection Active Pen for Apple iPad Pro 11" 4/3/2/1, iPad Pro 12.9" 6/5/4/3, iPad Air 4/5, iPad Mini 6 [Compatibility]- ONLY compatible with iPad mini (6th generation), iPad Air (4th and 5th generation), iPad Pro 12.9-inch (3rd, 4th, 5th and 6th generation), iPad Pro 11-inch (1st, 2nd, 3rd and 4th generation), check and confirm your device before place the order (Note: If the pen doesn\'t charge, fully charge your iPad first then try charging the pen again) [Charging and Pairs Magnetically]- Charges wirelessly, attaches and pairs magnetically to the compatible iPad, this pen is a preferable alternative to the Apple Pencil 2nd Generation [Tilt Sensitivity & Pixel Precision]- Pixel-perfect precision and industry-leading low latency with tilt sensitivity making drawing, sketching, coloring, tak

In [9]:
print(SYSTEM_PROMPT)


I am building a RAG application. I have a collection of 50 chunks of text.
The RAG application will act as a shopping assistant that can answer questions about the stock of the products we have available.
I will provide all of the available products to you with IDs of each chunk.
I want you to come up with 30 questions to which the answers could be grounded in the chunk context.
As an output I need you to provide me the list of questions and the IDs of the chunks that could be used to answer them.
Also, provide an example answer to the question given the context of the chunks.
Also, provide the reason why you chose the chunks to answer the questions.
Construct 10 that could use multiple chunks in the answer.
Construct 15 questions that could use single chunk in the answer.
Also, include 5 questions that can't be answered with the available chunks.

<OUTPUT JSON SCHEMA>
{
  "type": "array",
  "items": {
    "type": "object",
    "properties": {
      "question": {
        "type": "stri

In [10]:
response = openai.chat.completions.create(
    model="gpt-4.1",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ]
)

print(response.choices[0].message.content)

```json
[
  {
    "question": "Which products are compatible with the iPad Air 5th generation?",
    "chunk_ids": ["B09QGNB537", "B0BF18F6R7", "B0BYYLJRHT"],
    "answer_example": "The ESR Case (B09QGNB537) and the KEEPRO Pencil 2nd Generation (B0BF18F6R7) are compatible with the iPad Air 5th generation. Additionally, the iPhone Charger Cord Lightning Cables (B0BYYLJRHT) are also compatible with this iPad model for charging.",
    "reasoning": "All referenced chunks mention compatibility with the iPad Air 5th generation, and together cover cases, styluses, and cables."
  },
  {
    "question": "Do you have any product that supports both 5GHz and 2.4GHz WiFi bands?",
    "chunk_ids": ["B0BZ5R7CVP"],
    "answer_example": "Yes, the Tenda A33 AX3000 WiFi 6 Extender (B0BZ5R7CVP) supports both 2.4GHz and 5GHz WiFi bands.",
    "reasoning": "This chunk explicitly specifies support for 2.4/5GHz Dual Band."
  },
  {
    "question": "Are there any Bluetooth earphones suitable for running or wor

In [11]:
import json

In [12]:
json_output = response.choices[0].message.content
json_output = json_output.replace("```json", "")
json_output = json_output.replace("```", "")
json_output = json_output.replace("// Single-chunk questions (15)", "")
json_output = json.loads(json_output)

In [13]:
len(json_output)

34

In [18]:
json_output

[{'question': 'Which products are compatible with the iPad Air 5th generation?',
  'chunk_ids': ['B09QGNB537', 'B0BF18F6R7', 'B0BYYLJRHT'],
  'answer_example': 'The ESR Case (B09QGNB537) and the KEEPRO Pencil 2nd Generation (B0BF18F6R7) are compatible with the iPad Air 5th generation. Additionally, the iPhone Charger Cord Lightning Cables (B0BYYLJRHT) are also compatible with this iPad model for charging.',
  'reasoning': 'All referenced chunks mention compatibility with the iPad Air 5th generation, and together cover cases, styluses, and cables.'},
 {'question': 'Do you have any product that supports both 5GHz and 2.4GHz WiFi bands?',
  'chunk_ids': ['B0BZ5R7CVP'],
  'answer_example': 'Yes, the Tenda A33 AX3000 WiFi 6 Extender (B0BZ5R7CVP) supports both 2.4GHz and 5GHz WiFi bands.',
  'reasoning': 'This chunk explicitly specifies support for 2.4/5GHz Dual Band.'},
 {'question': 'Are there any Bluetooth earphones suitable for running or workouts that are waterproof?',
  'chunk_ids': ['

In [14]:
points = qdrant_client.scroll(
    collection_name="Amazon-items-collection-00",
    scroll_filter=Filter(
        must=[
            FieldCondition(
                key="parent_asin",
                match=MatchValue(value="B0BYYLJRHT")
            )
        ]
    ),
    limit=100,
    with_payload=True,
    with_vectors=False
)[0]

In [15]:
points[0].payload

{'description': 'iPhone Charger Cord Lightning Cables, Original 2022 Upgraded [3Pack 3ft] Apple MFi Certified USB A Charging Cable for iPhone 13 12 11 Mini Pro XR Xs Max X SE 8 7 6 Plus iPad iPod AirPods - Black Original Apple Cable - iPhone charging cable with a reinforced joint design that has passed 30,000+ times bending tests for extra protection and durability. Insulation and precisely layer-welded connectors, which make the cable more durable and sturdier than normal iPhone charger cables but also flexible and tangle-free. Instant Responses - No error message pops up. Real high-speed iPhone charging cable with stable and efficient current output, you can plug it into laptop computers and in-car charging adapter. Super Fast Charge - The iPhone Lightning cable build-in four-core high-quality copper wires and multiple safety protections overcharge/stable current/automatic switching/battery protection to promote maximum signal quality and strength. You can enjoy faster charging [3.0A

In [ ]:
# pointsdef get_description

[Record(id=4, payload={'description': 'iPhone Charger Cord Lightning Cables, Original 2022 Upgraded [3Pack 3ft] Apple MFi Certified USB A Charging Cable for iPhone 13 12 11 Mini Pro XR Xs Max X SE 8 7 6 Plus iPad iPod AirPods - Black Original Apple Cable - iPhone charging cable with a reinforced joint design that has passed 30,000+ times bending tests for extra protection and durability. Insulation and precisely layer-welded connectors, which make the cable more durable and sturdier than normal iPhone charger cables but also flexible and tangle-free. Instant Responses - No error message pops up. Real high-speed iPhone charging cable with stable and efficient current output, you can plug it into laptop computers and in-car charging adapter. Super Fast Charge - The iPhone Lightning cable build-in four-core high-quality copper wires and multiple safety protections overcharge/stable current/automatic switching/battery protection to promote maximum signal quality and strength. You can enjoy

In [17]:
def get_description(parent_asin: str) -> str:

    points = qdrant_client.scroll(
        collection_name="Amazon-items-collection-00",
        scroll_filter=Filter(
            must=[
                FieldCondition(
                    key="parent_asin",
                    match=MatchValue(value=parent_asin)
                )
            ]
        ),
        limit=100,
        with_payload=True,
        with_vectors=False
    )[0]

    return points[0].payload["description"]

### Create Eval Dataset in Langsmith

In [19]:
removed_item = json_output.pop(28)

In [20]:
removed_item

{'question': 'Are there replacement ear tips available for Powerbeats Pro?',
 'chunk_ids': ['B0B14HTZ59'],
 'answer_example': 'Yes, the 8 Pairs Powerbeats Pro Ear Tips Buds (B0B14HTZ59) are compatible replacements, available in multiple sizes.',
 'reasoning': 'Chunk details compatibility and options for replacements.'}

In [21]:
client = Client(api_key=os.environ["LANGSMITH_API_KEY"])

In [24]:
dataset_name = "rag-evaluation-dataset"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Dataset for evaluating RAG pipeline"
)

In [25]:
for item in json_output:
    print(item["chunk_ids"])
    client.create_example(
        dataset_id=dataset.id,
        inputs={"question": item["question"]},
        outputs={
            "ground_truth": item["answer_example"],
            "reference_context_ids": item["chunk_ids"],
            "reference_descriptions": [get_description(id) for id in item["chunk_ids"]]
        }
    )

['B09QGNB537', 'B0BF18F6R7', 'B0BYYLJRHT']
['B0BZ5R7CVP']
['B0C6K1GQCF', 'B0CBMPG524', 'B0BNHVLF7G']
['B0B7495RL6']
['B0B6V8THVX']
['B09TNXY54Y']
['B0BZ5R7CVP']
['B09WCFC5D9']
['B08BR9X387', 'B0BB1YKXL1']
['B09XCKYXR8']
['B0CBMPG524', 'B0BNHVLF7G']
['B0BG5L2YLC']
['B0BGDQLZD2']
['B0BR8Q2PXN', 'B0B55TZLBR']
['B0BS15TRJ3', 'B0BNVKS9WH']
['B09Y39DSWR']
['B0B3MMP22L']
['B0BFPZGYLD']
['B0BLV7YH2W']
['B0CF57H28T']
['B0BV6PWVCG']
['B09XCKYXR8', 'B09LVX3XW2', 'B0C9XFF3CT', 'B0BG5L2YLC']
['B0BNN776VN']
['B0B55TZLBR']
['B0BP9Z159S']
['B0C142QS8X']
['B09LVX3XW2']
['B0CF1WM24K']
[]
[]
[]
[]
[]
